# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [31]:
import pandas as pd

In [32]:
# Load csv file into pandas data frame
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [33]:
# Sort dataframe by control and experimental
df = df.sort_values(by='group')

,timestamp,id,group,action
4093,2016-11-22 11:21:15.157489,812204,control,view
4600,2016-11-29 18:53:11.706968,450958,control,view
4593,2016-11-29 15:35:40.687896,607909,control,view
4592,2016-11-29 14:59:39.843041,502906,control,view
4591,2016-11-29 14:55:56.719228,375688,control,view


In [36]:
# View the number of results for each category
print(df['group'].value_counts())
print(df['action'].value_counts())

control       4264
experiment    3924
Name: group, dtype: int64
view     6328
click    1860
Name: action, dtype: int64


In [37]:
# Lets find out the range that this data starts and ends 

# Convert the 'timestamp' column to datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Find the start and end dates and times
start_date_time = df['timestamp'].min()
end_date_time = df['timestamp'].max()

print("Start Date and Time:", start_date_time)
print("End Date and Time:", end_date_time)

Start Date and Time: 2016-09-24 17:42:27.839496
End Date and Time: 2017-01-18 10:24:08.629327


In [38]:
# Lets see if the id column can give any meaningful information

df = df.sort_values(by='id')
df

,timestamp,id,group,action
4390,2016-11-26 14:38:47.682931,182988,experiment,view
5639,2016-12-14 13:19:00.940677,182994,control,view
5640,2016-12-14 13:20:29.564080,182994,control,click
376,2016-09-29 23:04:42.722325,183089,control,view
6568,2016-12-27 10:13:58.808014,183136,experiment,view
...,...,...,...,...
7389,2017-01-07 11:50:17.313666,937108,control,view
5254,2016-12-09 07:05:04.965876,937139,experiment,view
5255,2016-12-09 07:06:05.219267,937139,experiment,click
3493,2016-11-14 08:00:01.730193,937217,control,view


In [39]:
# Take a look for overlapping id's
print(df['id'].value_counts())

696456    2
286434    2
908198    2
883682    2
821557    2
         ..
736506    1
276298    1
902988    1
642893    1
196608    1
Name: id, Length: 6328, dtype: int64


In [40]:
# This will give us a true representation of the control vs expiremental clicks and views. 
clicks_id = set(df[df.action=='click']['id'].unique())
views_id = set(df[df.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber of clickers: {}".format(len(views_id), len(clicks_id)))
print("Number of Viewers who didn't click: {}".format(len(views_id-clicks_id)))
print("Number of Clickers who didn't view: {}".format(len(clicks_id-views_id)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [41]:
exp_ids = set(df[df.group=='experiment']['id'].unique())
control_ids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(exp_ids&control_ids)))

Overlap of experiment and control groups: 0


In [ ]:
# It seems that there are two categorical groups expiremental and control. 

# There also seems to be an action called view, which suggests 
# if a person viewed the content or clicked the content.These values are also categorical.

# There is a timestamp which we found out that the date runs from 2016-09-24 through 2017-01-18. 
# This is approxametly 4 months worth of data. 

# Lastly there is an id colum. This id column seem to be instances when either a view or click occured.
# There are id's that overlap meaning that there are milliseconds appart or seconds apart from eachother.
# They overlap because the person first viewed the content then a second later clicked the content. 

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [42]:
# first convert our categorical data into useful numeric data by counting instances of each group
df['count'] = 1
df.head()

,timestamp,id,group,action,count
4390,2016-11-26 14:38:47.682931,182988,experiment,view,1
5639,2016-12-14 13:19:00.940677,182994,control,view,1
5640,2016-12-14 13:20:29.564080,182994,control,click,1
376,2016-09-29 23:04:42.722325,183089,control,view,1
6568,2016-12-27 10:13:58.808014,183136,experiment,view,1


In [44]:
# Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [45]:
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [51]:
# Since we have categorical values we should use Welch's t-test 
import numpy as np
import scipy.stats as stats

# Calculate Welch's t-test statistic and calculate the p-value
welch_t_test = stats.ttest_ind(control.click, experiment.click)
P_value = welch_t_test[1]/2
print("P_value: ", P_value)

P_value:  0.0044125494574791466


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [52]:
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [54]:
n = len(experiment)
p = control_rate
variance = n * p * (1-p)
std = np.sqrt(variance)
print(std)

24.568547907005815


In [55]:
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [56]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
# Comment: Yes, while the p-value is slightly lower, 
# both would lead to confident rejection of the null hypothesis. 
# The experimental page appears to be a more effective design.S

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.